<h1>Exploiting Tensorflow Lattice to solve ARC</h1>

## Introduction
The whole story started with twitter, from where I came to know about this competition. Most probably this competition was launched by Kaggle after a very long gap. This competition was discussed among various individuals in the data science community on twitter. I guessed I would approach the solution through some machine learning algorithms or deep learning. But suddenly after two days of an announcement of this competition, I came across a tweet on my timeline by an anonymous user saying **"Its a coincidence that TensorFlow released its new library TensorFlow Lattice on the same day this competition was announced by Kaggle**".<br>
The first time when I heard about TensorFlow lattice my reaction was...
![meme](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.OdvSEFavLCxXLlnBuP0NYQHaEK%26pid%3DApi&f=1)


## An important tweet by François Chollet
Below are some of the screenshots of twitter post by François Chollet(@fchollet) which would briefly explain the main objective of this competition.<br>
> **One interesting thing about the ARC competition is that it serves to highlight how people who use deep learning often have little idea of what deep learning actually does, and when they should be using it or not**

> **DL is applicable when you're doing *pattern recognition*: when you have data that lies on a smooth manifold, along which samples can be interpolated. And you're going to need a dense sampling of your manifold as training data in order to fit a parametric approximation of it**

> **Generalization in deep learning is interpolation along a latent manifold (or rather a learned approximation of it). It has little to do with your model itself and everything to do with the natural organization of your data**

> **Differentiability & minibatch SGD are the strengths of DL: besides making the learning practically tractable, the smoothness & continuity of the function & the incrementality of its fitting work great to learn to approximate latent manifold. But its strengths are also its limits**

> **The whole setup breaks down when you are no longer doing pattern recognition -- when you no longer have a latent manifold (any kind of discrete problem) or no longer have a dense sampling of it. Or when your manifold changes over time.**

> **This isn't complicated**

<br>
Link to the twitter post - https://twitter.com/fchollet/status/1234789789309652992

## What is Tensorflow Lattice?

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('ABBnNjbjv2Q', width=800, height=450)

## TensorFlow Lattice: Flexible, controlled and interpretable ML 
**by Mahdi Milani Fard, Software Engineer, Google Research**

Link to the original blog - https://blog.tensorflow.org/2020/02/tensorflow-lattice-flexible-controlled-and-interpretable-ML.html <br><br>
### Intro
Most ML practitioners have encountered the typical scenario where the training data looks very different from the run-time queries on which the model is evaluated. As a result, flexible ML solutions such as DNNs or forests that rely solely on the training dataset often act unexpectedly and even wildly in parts of the input space not covered by the training and validation datasets. This behaviour is especially problematic in cases where important policy or fairness constraints can be violated.<br><br>
Even though common forms of regularization can result in more sensible extrapolation, standard regularizers cannot guarantee reasonable model behaviour across the entire input space, especially with high-dimensional inputs. Switching to simpler models with more controlled and predictable behaviour can come at a severe cost to the model accuracy.<br><br>
TF Lattice makes it possible to keep using flexible models, but provides several options to inject domain knowledge into the learning process through semantically meaningful common-sense or policy-driven shape constraints.
 For example, you can specify that the model output should be monotonically increasing with respect to a given input. These extra pieces of domain knowledge can help the model learn beyond just the training dataset and makes it behave in a manner controlled and expected by the user.<br><br>
 A **lattice** is an interpolated look-up table that can approximate arbitrary input-output relationships in your data.
![lattice](https://www.tensorflow.org/lattice/images/model_comparison.png?) 

## Into the Problem

## The most difficult part of this Problem...
The most difficult part while approaching the solution was the implementation of TensorFlow Lattice itself. The implementation guide provided by TensorFlow was very helpful to understand how to use it over tabular data, but what for 2d-matrix data like this?. I went through a deep analysis of how TensorFlow lattice works and then found out a way to implement it. It took almost three weeks for me to implement the Lattice model. I was stressed by the thought of whether it would work successfully or not at the end. Although I also can't deny the fact that implementing Lattice over 2d-matrix was much easier than it looked.<br>
Trust me I tried my hardest to make it successful each time I failed just to prove that I did not waste my time for three weeks.<br>
![meme](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmedia.giphy.com%2Fmedia%2FQOaTohH90fuEM%2Fgiphy.gif&f=1&nofb=1)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd

import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors
print(os.listdir("../input/"))
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [ ]:
pip install -q tensorflow-lattice

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import logging
import tensorflow_lattice as tfl
import sys
from tensorflow import keras
tf.compat.v1.set_random_seed(123)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
logging.disable(sys.maxsize)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)


## Exploring the data

In [ ]:
from pathlib import Path

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

In [ ]:
training_tasks = sorted(os.listdir(training_path))
print(training_tasks[:3])

In [ ]:
task_file = str(training_path / '00d62c1b.json')

with open(task_file, 'r') as f:
    task = json.load(f)

print(task.keys())

In [ ]:
n_train_pairs = len(task['train'])
n_test_pairs = len(task['test'])

print(f'task contains {n_train_pairs} training pairs')
print(f'task contains {n_test_pairs} test pairs')

In [ ]:
display(task['train'][0]['input'])
display(task['train'][0]['output'])

In [ ]:
def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 4, figsize=(15,15))
    axs[0].imshow(task['train'][2]['input'], cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Train Input')
    axs[1].imshow(task['train'][2]['output'], cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Train Output')
    axs[2].imshow(task['test'][0]['input'], cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Test Input')
    axs[3].imshow(task['test'][0]['output'], cmap=cmap, norm=norm)
    axs[3].axis('off')
    axs[3].set_title('Test Output')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_task(task)

In [ ]:
def extract_data(files):
    train_x= []
    train_y = []
    test_x= []
    test_y = []
    for file in files:
        train_temp_x= []
        train_temp_y = []
        test_temp_x= []
        test_temp_y = []
        task_file = str(training_path / file)
        with open(task_file, 'r') as f:
            task = json.load(f)
        for c in range(len(task['train'])):
            train_temp_x.append(np.asarray(task['train'][c]['input']))
            train_temp_y.append(np.asarray(task['train'][c]['output']))
        for c in range(len(task['test'])):
            test_temp_x.append(np.asarray(task['test'][c]['input']))
            test_temp_y.append(np.asarray(task['test'][c]['output']))
        train_x.append(train_temp_x)
        train_y.append(train_temp_y)
        test_x.append(test_temp_x)
        test_y.append(test_temp_y)
    return train_x, train_y, test_x, test_y    

In [ ]:
train_x, train_y, test_x, test_y  = extract_data(training_tasks)

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 4, figsize=(15,15))
axs[0].imshow(train_x[0][1], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('Train Input')
axs[1].imshow(train_y[0][1], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('Train Output')
axs[2].imshow(test_x[0][0], cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('Test Input')
axs[3].imshow(test_y[0][0], cmap=cmap, norm=norm)
axs[3].axis('off')
axs[3].set_title('Test Output')
plt.tight_layout()

## Construction of the Lattice Neural Network

In [ ]:
NUM_EPOCHS = 2000
BATCH_SIZE = 64
LEARNING_RATE=0.001

In [ ]:
def reshapeData(X_, y_):
    X_ = X_.reshape(X_.shape[0],X_.shape[1],X_.shape[2],1)
    y_ = y_.reshape(y_.shape[0],y_.shape[1],y_.shape[2],1)
    return X_,y_

In [ ]:
def InitializeSession():
    tf.compat.v1.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.set_random_seed(123)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
def Lattice_Model(input_data, target_data, use_lim):
    
    # We are going to have 2-d embedding as one of lattice inputs.
    lattice_sizes = [5, 4, 3, 2, 3, 4, 5]
    
    input_ = tf.keras.layers.Input(shape=(input_data.shape[1], input_data.shape[2],1), name='Input')
    conv_1 = tf.keras.layers.Conv2D(128, 
                                    (3,3),
                                    activation='relu',
                                    padding='same', 
                                    name='conv_1')
    activation = tf.keras.layers.LeakyReLU()
    conv_2 = tf.keras.layers.Conv2D(128, 
                                    (1,1),
                                    activation='relu', 
                                    name='conv_2')
    conv_3 = tf.keras.layers.Conv2D(128,
                                    5,
                                    activation='relu',
                                    padding='same',
                                    name='conv_3')
    dropout = tf.keras.layers.Dropout(0.4,
                                      name='Dropout')
    
    flatten = tf.keras.layers.Flatten(name='Flatten')
    
    dense_1 = tf.keras.layers.Dense(target_data.shape[1]*target_data.shape[2]*7,
                                    name='Dense')
    
    reshape = tf.keras.layers.Reshape((target_data.shape[1],target_data.shape[2],7))
    if use_lim == 1:
        lattice = tfl.layers.Lattice(
                                    lattice_sizes=lattice_sizes,
                                    monotonicities=['none']*7,
                                    units=target_data.shape[2],
                                    output_min=target_data.min(),
                                    output_max=target_data.max(),
                                    name = 'Lattice'
                                    )
    else:
        lattice = tfl.layers.Lattice(
                                    lattice_sizes=lattice_sizes,
                                    monotonicities=['none']*7,
                                    units=target_data.shape[2],
                                    name = 'Lattice'
                                    )
    #model_1 = tf.keras.models.Sequential()
    model = (conv_1)(input_)
    model = (activation)(model)
    model = (conv_2)(model)
    model = (activation)(model)
    model = (dropout)(model)
    model = (conv_3)(model)
    model = (activation)(model)   
    model = (dropout)(model)
    model = (flatten)(model)
    model = (dropout)(model)
    model = (dense_1)(model)
    model = (activation)(model)
    model = (reshape)(model)
    model = (lattice)(model)
    #model.build()
    
    return tf.keras.models.Model(input_,model)

In [ ]:
def ModelFit(model, X_, y_, use_lim):
    if use_lim==1:
        loss_ = tf.keras.losses.mean_squared_error
    else:
        loss_ = tf.keras.losses.binary_crossentropy
    model.compile(
        loss=loss_,
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    )

    model.fit(
        X_,
        y_,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        validation_split=0.2,
        verbose=0
    )
    return model

## Note
what is this **use_lim** parameter? <br> Before Explaining this kernel, I found that some the some the problems requires **output_max**, **output_min** with optimization of **means_squared_error**, otherwise both **output_max**, **output_min**  as None with optimization of **binary_crossentropy** to obtain better results.

## Problem - 1

In [ ]:
train_x_1, train_y_1 = reshapeData(np.array(train_x[0]),np.array(train_y[0]))

In [ ]:
InitializeSession()
Model_1 = Lattice_Model(train_x_1, train_y_1,0)
Model_1.summary()

In [ ]:
Model_1 = ModelFit(Model_1,train_x_1, train_y_1,0)
Model_1.evaluate(train_x_1, train_y_1)

In [ ]:
pred_1 = Model_1.predict(np.array(test_x[0]).reshape(1,np.array(test_x[0]).shape[1],np.array(test_x[0]).shape[2],1))
pred_1 = pred_1.reshape(np.array(test_y[0]).shape[1],np.array(test_y[0]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[0][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[0][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_1), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout() 

Well the result is not so bad at all. But as you could see that I have rotated the predicted matrix by 90 degrees to match the desired result. We would be discussing that later in this notebook. Let's see how our lattice model performs to solve other problems.

## Problem - 2

In [ ]:
train_x_2, train_y_2 = reshapeData(np.array(train_x[2]),np.array(train_y[2]))

In [ ]:
InitializeSession()
Model_2 = Lattice_Model(train_x_2, train_y_2,0)
Model_2 = ModelFit(Model_2,train_x_2, train_y_2,0)
Model_2.evaluate(train_x_2, train_y_2)

In [ ]:
pred_2 = Model_2.predict(np.array(test_x[2]).reshape(1,np.array(test_x[2]).shape[1],np.array(test_x[2]).shape[2],1))
pred_2 = pred_2.reshape(np.array(test_y[2]).shape[1],np.array(test_y[2]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[2][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[2][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_2), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

The predicted results have smaller squares, but symmetry is maintained in the predicted output. Let us see what happens if we use mean_squared error for optimization.

In [ ]:
InitializeSession()
Model_2_1 = Lattice_Model(train_x_2, train_y_2,1)
Model_2_1 = ModelFit(Model_2_1,train_x_2, train_y_2,1)
Model_2_1.evaluate(train_x_2, train_y_2)

In [ ]:
pred_2_1 = Model_2_1.predict(np.array(test_x[2]).reshape(1,np.array(test_x[2]).shape[1],np.array(test_x[2]).shape[2],1))
pred_2_1 = pred_2_1.reshape(np.array(test_y[2]).shape[1],np.array(test_y[2]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[2][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[2][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_2_1), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

Looks like the symmetry is disturbed this time.

## Problem - 3

In [ ]:
train_x_3, train_y_3 = reshapeData(np.array(train_x[4]),np.array(train_y[4]))

In [ ]:
InitializeSession()
Model_3 = Lattice_Model(train_x_3, train_y_3,1)
Model_3 = ModelFit(Model_3,train_x_3, train_y_3,1)
Model_3.evaluate(train_x_3, train_y_3)

In [ ]:
pred_3 = Model_3.predict(np.array(test_x[4]).reshape(1,np.array(test_x[4]).shape[1],np.array(test_x[4]).shape[2],1))
pred_3 = pred_3.reshape(np.array(test_y[4]).shape[1],np.array(test_y[4]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[4][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[4][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_3), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

Looks like the model did not perform much well this time....

## Problem - 4

In [ ]:
train_x_4, train_y_4 = reshapeData(np.array(train_x[5]),np.array(train_y[5]))

In [ ]:
InitializeSession()
Model_4 = Lattice_Model(train_x_4, train_y_4,0)
Model_4 = ModelFit(Model_4,train_x_4, train_y_4,0)
Model_4.evaluate(train_x_4, train_y_4)

In [ ]:
pred_4 = Model_4.predict(np.array(test_x[5]).reshape(1,np.array(test_x[5]).shape[1],np.array(test_x[5]).shape[2],1))
pred_4 = pred_4.reshape(np.array(test_y[5]).shape[1],np.array(test_y[5]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[5][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[5][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_4), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

It kind of looks like found a pattern, but as you can see the inverted T section should have black color and the corner square should have blue while the reverse has happened.

## Problem - 5

In [ ]:
train_x_5, train_y_5 = reshapeData(np.array(train_x[6]),np.array(train_y[6]))

In [ ]:
InitializeSession()
Model_5 = Lattice_Model(train_x_5, train_y_5,1)
Model_5 = ModelFit(Model_5,train_x_5, train_y_5,1)
Model_5.evaluate(train_x_5, train_y_5)

In [ ]:
pred_5 = Model_5.predict(np.array(test_x[6]).reshape(1,np.array(test_x[6]).shape[1],np.array(test_x[6]).shape[2],1))
pred_5 = pred_5.reshape(np.array(test_y[6]).shape[1],np.array(test_y[6]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[6][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[6][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(np.rot90(pred_5)), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

No, complains this time...

## Problem - 6

In [ ]:
train_x_6, train_y_6 = reshapeData(np.array(train_x[9]),np.array(train_y[9]))

In [ ]:
InitializeSession()
Model_6 = Lattice_Model(train_x_6, train_y_6,1)
Model_6 = ModelFit(Model_6,train_x_6, train_y_6,1)
Model_6.evaluate(train_x_6, train_y_6)

In [ ]:
pred_6 = Model_6.predict(np.array(test_x[9]).reshape(1,np.array(test_x[9]).shape[1],np.array(test_x[9]).shape[2],1))
pred_6 = pred_6.reshape(np.array(test_y[9]).shape[1],np.array(test_y[9]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[9][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[9][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_6), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

In this case, the model is understanding the pattern like a bar with the least height would be yellow, bar with the highest height would be blue, the second-highest would be red and third would be green. But, it does not produce the results keeping the respect of the order of the given input.

## Problem - 7

In [ ]:
train_x_7, train_y_7 = reshapeData(np.array(train_x[10]),np.array(train_y[10]))

In [ ]:
InitializeSession()
Model_7 = Lattice_Model(train_x_7, train_y_7,1)
Model_7 = ModelFit(Model_7,train_x_7, train_y_7,1)
Model_7.evaluate(train_x_7, train_y_7)

In [ ]:
pred_7 = Model_7.predict(np.array(test_x[10]).reshape(1,np.array(test_x[10]).shape[1],np.array(test_x[10]).shape[2],1))
pred_7 = pred_7.reshape(np.array(test_y[10]).shape[1],np.array(test_y[10]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[10][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[10][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_7), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

Guess, this did not work well...

## Problem - 8

In [ ]:
train_x_8, train_y_8 = reshapeData(np.array(train_x[11]),np.array(train_y[11]))

In [ ]:
InitializeSession()
Model_8 = Lattice_Model(train_x_8, train_y_8,1)
Model_8 = ModelFit(Model_8,train_x_8, train_y_8,1)
Model_8.evaluate(train_x_8, train_y_8)

In [ ]:
pred_8 = Model_8.predict(np.array(test_x[11]).reshape(1,np.array(test_x[11]).shape[1],np.array(test_x[11]).shape[2],1))
pred_8 = pred_8.reshape(np.array(test_y[11]).shape[1],np.array(test_y[11]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[11][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[11][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_8), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

Not bad...

## Problem - 9

In [ ]:
train_x_9, train_y_9 = reshapeData(np.array(train_x[14]),np.array(train_y[14]))

In [ ]:
InitializeSession()
Model_9 = Lattice_Model(train_x_9, train_y_9,1)
Model_9 = ModelFit(Model_9,train_x_9, train_y_9,1)
Model_9.evaluate(train_x_9, train_y_9)

In [ ]:
pred_9 = Model_9.predict(np.array(test_x[14]).reshape(1,np.array(test_x[14]).shape[1],np.array(test_x[14]).shape[2],1))
pred_9 = pred_9.reshape(np.array(test_y[14]).shape[1],np.array(test_y[14]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[14][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[14][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_9), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

## Problem - 10

In [ ]:
train_x_10, train_y_10 = reshapeData(np.array(train_x[15]),np.array(train_y[15]))

In [ ]:
InitializeSession()
Model_10 = Lattice_Model(train_x_10, train_y_10,1)
Model_10 = ModelFit(Model_10,train_x_10, train_y_10,1)
Model_10.evaluate(train_x_10, train_y_10)

In [ ]:
pred_10 = Model_10.predict(np.array(test_x[15]).reshape(1,np.array(test_x[15]).shape[1],np.array(test_x[15]).shape[2],1))
pred_10 = pred_10.reshape(np.array(test_y[15]).shape[1],np.array(test_y[15]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[15][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[15][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(np.rot90(pred_10)), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

## Problem - 11

In [ ]:
train_x_11, train_y_11 = reshapeData(np.array(train_x[16]),np.array(train_y[16]))

In [ ]:
InitializeSession()
Model_11 = Lattice_Model(train_x_11, train_y_11,1)
Model_11 = ModelFit(Model_11,train_x_11, train_y_11,1)
Model_11.evaluate(train_x_11, train_y_11)

In [ ]:
pred_11 = Model_11.predict(np.array(test_x[16]).reshape(1,np.array(test_x[16]).shape[1],np.array(test_x[16]).shape[2],1))
pred_11 = pred_11.reshape(np.array(test_y[16]).shape[1],np.array(test_y[16]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[16][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[16][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_11), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

## Problem - 12

In [ ]:
train_x_12, train_y_12 = reshapeData(np.array(train_x[19]),np.array(train_y[19]))

In [ ]:
InitializeSession()
Model_12 = Lattice_Model(train_x_12, train_y_12,1)
Model_12 = ModelFit(Model_12,train_x_12, train_y_12,1)
Model_12.evaluate(train_x_12, train_y_12)

In [ ]:
pred_12 = Model_12.predict(np.array(test_x[19]).reshape(1,np.array(test_x[19]).shape[1],np.array(test_x[19]).shape[2],1))
pred_12 = pred_12.reshape(np.array(test_y[19]).shape[1],np.array(test_y[19]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[19][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[19][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_12), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

Let's try to solve this problem by optimizing binary_crossentropy rather than mean_squared_error

In [ ]:
InitializeSession()
Model_12_1 = Lattice_Model(train_x_12, train_y_12,0)
Model_12_1 = ModelFit(Model_12_1,train_x_12, train_y_12,0)
Model_12_1.evaluate(train_x_12, train_y_12)

In [ ]:
pred_12_1 = Model_12_1.predict(np.array(test_x[19]).reshape(1,np.array(test_x[19]).shape[1],np.array(test_x[19]).shape[2],1))
pred_12_1 = pred_12_1.reshape(np.array(test_y[19]).shape[1],np.array(test_y[19]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[19][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[19][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(pred_12_1), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

While optimizing mean_squared_error the model is only able to detect which part of the incomplete pattern is needed to be filled up but is not able to answer which color it would be, while in case of binary_crossentrophy it is answer with entire pattern but is not able to colorize the pattern.

## Problem - 13

In [ ]:
train_x_13, train_y_13 = reshapeData(np.array(train_x[21]),np.array(train_y[21]))

In [ ]:
InitializeSession()
Model_13 = Lattice_Model(train_x_13, train_y_13,1)
Model_13 = ModelFit(Model_13,train_x_13, train_y_13,1)
Model_13.evaluate(train_x_13, train_y_13)

In [ ]:
pred_13 = Model_13.predict(np.array(test_x[21]).reshape(1,np.array(test_x[21]).shape[1],np.array(test_x[21]).shape[2],1))
pred_13 = pred_13.reshape(np.array(test_y[21]).shape[1],np.array(test_y[21]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[21][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[21][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow(np.rot90(np.rot90(pred_13.T)), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

The problem looks like a puzzle-solving task, and it seems that our lattice model is pretty weak in solving these types of tasks.

## Problem - 14

In [ ]:
train_x_14, train_y_14 = reshapeData(np.array(train_x[25]),np.array(train_y[25]))

In [ ]:
InitializeSession()
Model_14 = Lattice_Model(train_x_14, train_y_14,0)
Model_14 = ModelFit(Model_14,train_x_14, train_y_14,0)
Model_14.evaluate(train_x_14, train_y_14)

In [ ]:
pred_14 = Model_14.predict(np.array(test_x[25]).reshape(1,np.array(test_x[25]).shape[1],np.array(test_x[25]).shape[2],1))
pred_14 = pred_14.reshape(np.array(test_y[25]).shape[1],np.array(test_y[25]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[25][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[25][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_14), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

The task in this problem was to highlight out the superimposed section of the half divided input image. Yes, our model failed here also...

## Problem - 15

In [ ]:
train_x_15, train_y_15 = reshapeData(np.array(train_x[26]),np.array(train_y[26]))

In [ ]:
InitializeSession()
Model_15 = Lattice_Model(train_x_15, train_y_15,1)
Model_15 = ModelFit(Model_15,train_x_15, train_y_15,1)
Model_15.evaluate(train_x_15, train_y_15)

In [ ]:
pred_15 = Model_15.predict(np.array(test_x[26]).reshape(1,np.array(test_x[26]).shape[1],np.array(test_x[26]).shape[2],1))
pred_15 = pred_15.reshape(np.array(test_y[26]).shape[1],np.array(test_y[26]).shape[2])

In [ ]:
cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(test_x[26][0], cmap=cmap, norm=norm)
axs[0].axis('off')
axs[0].set_title('test Input')
axs[1].imshow(test_y[26][0], cmap=cmap, norm=norm)
axs[1].axis('off')
axs[1].set_title('test Output')
axs[2].imshow((pred_15), cmap=cmap, norm=norm)
axs[2].axis('off')
axs[2].set_title('predicted output')
plt.tight_layout()

## Conclusion

* TF Lattice is not the absolute solution 
* It failed to solve puzzle-solving tasks and task involved combination of two inputs as one input.
* I need to rotate the predicted matrix or Transpose it to get the desired result, so that is some-what a bad behavior shown by our model.
* To solve all the 15 tasks we have used the same Neural Network model and same Lattice structure, same learning rate, and the same number of iterations, so I believe some unsolvable problems could have been solved with a different NN structure or different Lattice structure.

* For solving bigger size matrix problems like in case of problem-11, a higher dimension lattice could have been a better choice, while training it through a higher number of iterations.
* I have only approached the tasks that have uniform matrix shape for training, for non-uniform distribution I hope concepts like cellular automata, genetic algorithms and many other concepts discussed in the Notebook section of the competition would definitely help.

<h2>Please UpVote and share if you like this notebook or if this notebook was informative to you by some means. Also, let me know your opinions and suggestions in the comment section below.</h2>
![memes](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.1lFiVY2gxBQobWgyRc9AbAHaHa%26pid%3DApi&f=1)

## Thank You....